In [1]:
# Importing required libraries
import simpledbf
import pandas as pd
import geopandas as gpd
import numpy as np
import h3
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, mapping
from shapely.ops import unary_union
import folium
import os

PyTables is not installed. No support for HDF output.


In [2]:
# Data Loading and cleaning

#OD dataset loading and cleaning
dbf = simpledbf.Dbf5('ucl-mres-thesis/OD2023 DATASET-Metro SP/Banco2023_divulgacao_190225.dbf')


In [3]:
df = dbf.to_dataframe()

In [4]:
# List of columns to check for zero values
columns_to_check = [
    'CO_O_X', 'CO_O_Y', 'CO_D_X', 'CO_D_Y'
]

# Keeping only rows where all these columns are non-zero
cleaned_df = df[(df[columns_to_check] != 0).all(axis=1)]

output_dir = "outputs"
os.makedirs(output_dir, exist_ok=True)

cleaned_df.to_csv(os.path.join(output_dir, 'cleandbf.csv'), index=False)

In [5]:
#Shapefile loading and cleaning
shapefile_path = "ucl-mres-thesis/Municipal boundaries-ShapeFiles/Municipios_2023.shp" 
gdf = gpd.read_file(shapefile_path)

In [6]:
# Data Processing

# Ensuring the shapefile is in WGS84 (lat/lon) for H3
if gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(epsg=4326)

gdf_exploded = gdf.explode(index_parts=True)

def polygon_to_h3(poly, resolution=9):
    geo_json = mapping(poly)
    if geo_json["type"] == "MultiPolygon":
        h3_indexes = set()
        for polygon in poly.geoms:
            poly_geo = mapping(polygon)
            poly_geo["coordinates"] = [
                [[y, x] for x, y in ring] for ring in poly_geo["coordinates"][0]
            ]
            h3_indexes.update(
                h3.polyfill(poly_geo, resolution, geo_json_conformant=True)
            )
        return list(h3_indexes)
    elif geo_json["type"] == "Polygon":
        geo_json["coordinates"] = [
            [[y, x] for x, y in ring] for ring in geo_json["coordinates"]
        ]
        return list(h3.polyfill(geo_json, resolution, geo_json_conformant=True))
    else:
        raise ValueError("Unsupported geometry type")


In [7]:
all_h3_indexes = set()
for poly in gdf_exploded.geometry:
    h3_cells = polygon_to_h3(poly, resolution=9)
    all_h3_indexes.update(h3_cells)

hex_polygons = []
for h3_index in all_h3_indexes:
    boundary = h3.h3_to_geo_boundary(h3_index, geo_json=True)
    hex_polygons.append(Polygon([(lon, lat) for lat, lon in boundary]))

hex_gdf = gpd.GeoDataFrame(
    {"h3_index": list(all_h3_indexes), "geometry": hex_polygons}, crs="EPSG:4326"
)

h3_dir = os.path.join(output_dir, "H3_res9")
os.makedirs(h3_dir, exist_ok=True)
output_path = os.path.join(h3_dir, "Municipios_2023_h3_res9.shp")
hex_gdf.to_file(output_path)

In [8]:
# Geting the centroid of the study area for map center
center = hex_gdf.geometry.union_all().centroid
m = folium.Map(location=[center.y, center.x], zoom_start=12)

# Add hexagon boundaries to the map
for _, row in hex_gdf.iterrows():
    geom = row.geometry
    folium.Polygon(
        locations=[(y, x) for x, y in zip(*geom.exterior.coords.xy)],
        color="blue",
        fill=False,
    ).add_to(m)

In [9]:
# Creating GEO points from the OD DataFrame

# Assigning the original CRS: EPSG:22523 (São Paulo - UTM 23S, meters)
projected_crs = "EPSG:22523"

# Create geometry columns for origins and destinations
from shapely.geometry import Point
df_with_geom = cleaned_df.copy()
df_with_geom["origin_geom"] = df_with_geom.apply(lambda row: Point(row["CO_O_X"], row["CO_O_Y"]), axis=1)
df_with_geom["dest_geom"] = df_with_geom.apply(lambda row: Point(row["CO_D_X"], row["CO_D_Y"]), axis=1)

# Create GeoDataFrames using the starting CRS
gdf_origin = gpd.GeoDataFrame(df_with_geom.copy(), geometry="origin_geom", crs=projected_crs)
gdf_dest = gpd.GeoDataFrame(df_with_geom.copy(), geometry="dest_geom", crs=projected_crs)

# REPROJECT BOTH TO EPSG:4326 for spatial matching with H3 grid
gdf_origin = gdf_origin.to_crs(epsg=4326)
gdf_dest = gdf_dest.to_crs(epsg=4326)

In [10]:
# Spatial joins to assign h3 hex indices

origin_join = gpd.sjoin(
    gdf_origin, hex_gdf[["h3_index", "geometry"]], how="left", predicate="within"
)
df_with_geom["h3_origin"] = origin_join["h3_index"].values

dest_join = gpd.sjoin(
    gdf_dest, hex_gdf[["h3_index", "geometry"]], how="left", predicate="within"
)
df_with_geom["h3_dest"] = dest_join["h3_index"].values


# Making cleaned_df a true copy to allow safe assignment
cleaned_df = cleaned_df.copy()

cleaned_df.loc[:, "h3_origin"] = df_with_geom["h3_origin"].values
cleaned_df.loc[:, "h3_dest"] = df_with_geom["h3_dest"].values

df_with_geom.to_csv(os.path.join(output_dir, 'od_with_h3.csv'), index=False)

In [11]:
# Defining 15-minute time windows

def get_quarter(hour, minute):
    if 0 <= minute < 15:
        return f"{hour:02d}:00–{hour:02d}:14"
    elif 15 <= minute < 30:
        return f"{hour:02d}:15–{hour:02d}:29"
    elif 30 <= minute < 45:
        return f"{hour:02d}:30–{hour:02d}:44"
    else:
        return f"{hour:02d}:45–{hour:02d}:59"


cleaned_df.loc[:, "quarter_departure"] = cleaned_df.apply(
    lambda row: get_quarter(row["H_SAIDA"], row["MIN_SAIDA"]), axis=1
)
cleaned_df.loc[:, "quarter_arrival"] = cleaned_df.apply(
    lambda row: get_quarter(row["H_CHEG"], row["MIN_CHEG"]), axis=1
)

In [12]:
cleaned_df.to_csv(os.path.join(output_dir, 'cleandbf.csv'), index=False)

In [13]:
# Transforming 15-minute time windows into indices (0-95)
def time_window_to_index(time_window):
    # Replace en dash and em dash with hyphen
    time_window = time_window.replace('–', '-').replace('—', '-')
    # Split on hyphen
    parts = time_window.split('-')
    start_time = parts[0].strip()
    # Check if time is in HH:MM format
    if ':' in start_time:
        hour, minute = map(int, start_time.split(':'))
    else:
        # If only hour is present, minute = 0
        hour = int(start_time)
        minute = 0
    return hour * 4 + minute // 15


cleaned_df.loc[:, 'departure_idx'] = cleaned_df['quarter_departure'].apply(time_window_to_index)
cleaned_df.loc[:, 'arrival_idx'] = cleaned_df['quarter_arrival'].apply(time_window_to_index)

In [ ]:
# Person-Time_Trip into (PTTI) dataframe creation

# Filtering adults and converting hex IDs to string type
adults_df = cleaned_df[cleaned_df['IDADE'] >= 18].copy()
adults_df['h3_origin'] = adults_df['h3_origin'].astype(str)
adults_df['h3_dest'] = adults_df['h3_dest'].astype(str)
adults_df['MOTIVO_O'] = adults_df['MOTIVO_O'].astype(str)
adults_df['MOTIVO_D'] = adults_df['MOTIVO_D'].astype(str)
adults_df['FE_VIA'] = adults_df['FE_VIA'].astype(float)

# Creating time indices (ensure this uses the fixed function from earlier)
def time_window_to_index(time_window):
    time_window = str(time_window).replace('–', '-').replace('—', '-')
    start_time = time_window.split('-')[0].strip()
    if ':' in start_time:
        hour, minute = map(int, start_time.split(':'))
    else:
        hour, minute = int(start_time), 0
    return hour * 4 + minute // 15

adults_df['departure_idx'] = adults_df['quarter_departure'].apply(time_window_to_index)
adults_df['arrival_idx'] = adults_df['quarter_arrival'].apply(time_window_to_index)


In [16]:
# Initializing person_df with object dtype for hex IDs
adult_persons = adults_df['ID_PESS'].unique()
person_hex = pd.DataFrame(index=adult_persons, columns=range(96), dtype=object)
person_motive = pd.DataFrame(index=adult_persons, columns=range(96), dtype=object)
person_fevia = pd.DataFrame(index=adult_persons, columns=range(96), dtype=object)
person_hex[:] = None
person_motive[:] = None
person_fevia[:] = None


In [17]:
# Precomputing first trip info
first_trips = (
    adults_df.sort_values('departure_idx')
    .groupby('ID_PESS')
    .head(1)
    .set_index('ID_PESS')
    [['h3_origin', 'MOTIVO_O', 'departure_idx', 'FE_VIA']]
)
for person, row in first_trips.iterrows():
    dep_idx = row['departure_idx']
    if dep_idx > 0:
        person_hex.loc[person, :dep_idx-1] = row['h3_origin']
        person_motive.loc[person, :dep_idx-1] = row['MOTIVO_O']
        person_fevia.loc[person, :dep_idx-1] = row['FE_VIA']

# Filling pre-first-trip periods
trip_groups = adults_df.sort_values(['ID_PESS', 'departure_idx']).groupby('ID_PESS')
for person, trips in trip_groups:
    trips = trips.reset_index(drop=True)
    for i in range(len(trips)):
        arr = trips.loc[i, 'arrival_idx']
        if i < len(trips) - 1:
            next_dep = trips.loc[i + 1, 'departure_idx']
        else:
            next_dep = 96
        if arr >= next_dep:
            continue
        person_hex.loc[person, arr:next_dep-1] = trips.loc[i, 'h3_dest']
        person_motive.loc[person, arr:next_dep-1] = trips.loc[i, 'MOTIVO_D']
        person_fevia.loc[person, arr:next_dep-1] = trips.loc[i, 'FE_VIA']


In [18]:
# Filling empty slots with "transit"

person_hex = person_hex.replace([None, np.nan, ""], "transit")
person_motive = person_motive.replace([None, np.nan, ""], "transit")
person_fevia = person_fevia.replace([None, np.nan, ""], "transit")

        
# Combining into one MultiIndex DataFrame
hex_vals = person_hex.values
motive_vals = person_motive.values
fevia_vals = person_fevia.values

arrays = [np.repeat(range(96), 3), ['hexagon', 'motive', 'tripexpfactor'] * 96]
multi_columns = pd.MultiIndex.from_arrays(arrays, names=['quarter', 'info'])

combo = np.empty((len(adult_persons), 96 * 3), dtype=object)
combo[:, 0::3] = hex_vals
combo[:, 1::3] = motive_vals
combo[:, 2::3] = fevia_vals

final_df = pd.DataFrame(combo, columns=multi_columns, index=adult_persons).reset_index()
final_df.rename(columns={'index': 'ID_PESS'}, inplace=True)


/tmp/ipykernel_56642/1429428910.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  person_fevia = person_fevia.replace([None, np.nan, ""], "transit")


In [19]:
final_df.to_csv(os.path.join(output_dir,"OD_ADULTS_Timeindex_Motive_location_tripexpansionfactors.csv"), index=False)